In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


In [2]:
gs = pd.read_csv('E0.csv')
gs

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,2.28,1.50,1.95,1.98,1.95,1.97,NaN,NaN,1.92,1.95
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,2.63,-2.00,1.95,1.98,1.93,1.97,2.01,2.09,1.95,1.92
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.12,0.00,2.02,1.91,2.01,1.92,2.06,1.96,1.96,1.91
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.48,-1.75,2.01,1.92,2.00,1.91,2.14,1.93,2.00,1.86
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,1.71,-0.25,2.06,1.87,2.04,1.88,2.08,1.99,1.98,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,E0,18/02/2024,14:00,Sheffield United,Brighton,0,5,A,0,2,...,2.43,0.75,2.07,1.86,2.06,1.87,2.12,1.89,2.03,1.84
246,E0,18/02/2024,16:30,Luton,Man United,1,2,A,1,2,...,2.54,0.75,1.90,2.03,1.90,2.02,1.93,2.06,1.86,2.00
247,E0,19/02/2024,20:00,Everton,Crystal Palace,1,1,D,0,0,...,1.62,-0.75,2.05,1.75,2.10,1.81,2.14,1.83,2.06,1.80
248,E0,20/02/2024,19:30,Man City,Brentford,1,0,H,0,0,...,3.16,-2.00,1.90,2.03,1.89,2.02,1.95,2.06,1.88,2.00


In [3]:
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR', 'HS', 'AS','HST','AST', 'HR', 'AR','HC','AC']
#FTHG full time home goals
#FTAG full time away goals
# in this model i will not be putting in any data regarding odds let's analyze the matches purely on 
# game data
#FTR is full time result
#HS home shots
#AS away shots
# not gonna use odds rn
#HST home shots target
#AST away shots target
#'BWD', 'BWA', 'BWH' odds
#need also RED cards
#HR,  AR
#gonna encode team names
#FTR TO BE MAPPED 2 1 0 h

gs = gs[columns_req]
gs

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HR,AR,HC,AC
0,11/08/2023,Burnley,Man City,0,3,A,6,17,1,8,1,0,6,5
1,12/08/2023,Arsenal,Nott'm Forest,2,1,H,15,6,7,2,0,0,8,3
2,12/08/2023,Bournemouth,West Ham,1,1,D,14,16,5,3,0,0,10,4
3,12/08/2023,Brighton,Luton,4,1,H,27,9,12,3,0,0,6,7
4,12/08/2023,Everton,Fulham,0,1,A,19,9,9,2,0,0,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,18/02/2024,Sheffield United,Brighton,0,5,A,6,24,1,10,1,0,5,10
246,18/02/2024,Luton,Man United,1,2,A,22,21,4,9,0,0,8,6
247,19/02/2024,Everton,Crystal Palace,1,1,D,19,10,4,4,0,0,3,6
248,20/02/2024,Man City,Brentford,1,0,H,25,6,11,2,0,0,10,1


In [4]:
table = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS'))


pd.unique(gs['HomeTeam'])
#250 matches to process
pd.unique(gs['AwayTeam'])
avg_home_scored = gs.FTHG.sum() / 250
avg_away_scored = gs.FTAG.sum() / 250
avg_home_conceded = avg_away_scored
avg_away_conceded = avg_home_scored
print(avg_home_conceded)




1.484


In [9]:
len(gs[(gs['HomeTeam']=='Burnley')])

13

In [5]:
tms = pd.unique(gs["HomeTeam"])
tms.sort()
tms

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Liverpool', 'Luton', 'Man City', 'Man United', 'Newcastle',
       "Nott'm Forest", 'Sheffield United', 'Tottenham', 'West Ham',
       'Wolves'], dtype=object)

In [6]:
res_home = gs.groupby('HomeTeam')
res_away = gs.groupby('AwayTeam')

In [7]:
table.Team = tms
table.HGS = res_home.FTHG.sum().values
table.HGC = res_home.FTAG.sum().values
table.AGS = res_away.FTAG.sum().values
table.AGC = res_away.FTHG.sum().values
for i,t in enumerate(tms):
    table.iloc[i,3]= (table.iloc[i,1]/(len(gs[(gs['HomeTeam']==t)])))/avg_home_scored
    table.iloc[i,4]=(table.iloc[i,2]/len(gs[(gs['AwayTeam']==t)]))/avg_away_scored
    table.iloc[i,[7]]=(table.iloc[i,5]/len(gs[(gs['HomeTeam']==t)]))/avg_home_conceded
    table.iloc[i,[8]]=(table.iloc[i,6]/len(gs[(gs['AwayTeam']==t)]))/avg_away_conceded


In [8]:
gs
features = gs[['HomeTeam','AwayTeam', 'HR','AR', 'FTR']]
features

f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in features.iterrows():
    f_HAS.append(table[table['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table[table['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table[table['Team'] == row['AwayTeam']]['AAS'].values[0])
    f_ADS.append(table[table['Team'] == row['AwayTeam']]['ADS'].values[0])

features['HAS'] = f_HAS
features['HDS'] = f_HDS
features['AAS'] = f_AAS
features['ADS'] = f_ADS
features

/var/folders/xl/8ct_2v_x2bxcwb922dn6qd5m0000gn/T/ipykernel_25064/765010272.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['HAS'] = f_HAS
/var/folders/xl/8ct_2v_x2bxcwb922dn6qd5m0000gn/T/ipykernel_25064/765010272.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['HDS'] = f_HDS
/var/folders/xl/8ct_2v_x2bxcwb922dn6qd5m0000gn/T/ipykernel_25064/765010272.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,HomeTeam,AwayTeam,HR,AR,FTR,HAS,HDS,AAS,ADS
0,Burnley,Man City,1,0,A,0.573394,1.658719,1.516173,0.716743
1,Arsenal,Nott'm Forest,0,0,H,1.433486,0.617700,0.730009,1.146789
2,Bournemouth,West Ham,0,0,D,0.625521,1.041411,0.933029,1.190896
3,Brighton,Luton,0,0,H,1.242355,0.842318,0.898473,1.385703
4,Everton,Fulham,0,0,A,0.617502,0.777524,0.617700,1.242355
...,...,...,...,...,...,...,...,...,...
245,Sheffield United,Brighton,1,0,A,0.529287,1.866058,1.140369,1.102682
246,Luton,Man United,0,0,A,0.838038,1.140369,0.881194,0.705716
247,Everton,Crystal Palace,0,0,D,0.617502,0.777524,0.673854,1.102682
248,Man City,Brentford,0,0,H,1.367325,0.570185,0.730009,0.812309


In [15]:
table

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,30,28,1.433486,1.451379,11,11,0.6177,0.48518
1,Aston Villa,31,21,1.481269,1.088534,13,20,0.730009,0.882145
2,Bournemouth,12,21,0.625521,1.088534,17,29,1.041411,1.279111
3,Brentford,22,13,0.97036,0.730009,27,17,1.399544,0.812309
4,Brighton,26,22,1.242355,1.140369,15,25,0.842318,1.102682
5,Burnley,13,12,0.573394,0.673854,32,23,1.658719,1.099006
6,Chelsea,20,22,0.955657,1.140369,18,23,1.010782,1.014467
7,Crystal Palace,15,13,0.716743,0.673854,19,25,1.066936,1.102682
8,Everton,14,13,0.617502,0.730009,15,18,0.777524,0.860092
9,Fulham,23,11,1.014467,0.6177,15,26,0.777524,1.242355


In [9]:
#Function to transform FTR into numeric data type
def transformResult(row):
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return -1
    else:
        return 0

features["Result"] = features.apply(lambda row: transformResult(row),axis=1)
features

/var/folders/xl/8ct_2v_x2bxcwb922dn6qd5m0000gn/T/ipykernel_25064/1377873479.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["Result"] = features.apply(lambda row: transformResult(row),axis=1)


,HomeTeam,AwayTeam,HR,AR,FTR,HAS,HDS,AAS,ADS,Result
0,Burnley,Man City,1,0,A,0.573394,1.658719,1.516173,0.716743,-1
1,Arsenal,Nott'm Forest,0,0,H,1.433486,0.617700,0.730009,1.146789,1
2,Bournemouth,West Ham,0,0,D,0.625521,1.041411,0.933029,1.190896,0
3,Brighton,Luton,0,0,H,1.242355,0.842318,0.898473,1.385703,1
4,Everton,Fulham,0,0,A,0.617502,0.777524,0.617700,1.242355,-1
...,...,...,...,...,...,...,...,...,...,...
245,Sheffield United,Brighton,1,0,A,0.529287,1.866058,1.140369,1.102682,-1
246,Luton,Man United,0,0,A,0.838038,1.140369,0.881194,0.705716,-1
247,Everton,Crystal Palace,0,0,D,0.617502,0.777524,0.673854,1.102682,0
248,Man City,Brentford,0,0,H,1.367325,0.570185,0.730009,0.812309,1


In [25]:
X= features[['HAS','HDS','AAS','ADS','HR','AR']]
y = features['Result']

X_train, X_test, y_train, y_test = train_test_split( 
                        X,y,test_size = 0.3, random_state = 101) 



In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV

In [12]:

model = SVC() 
model.fit(X_train, y_train) 

predictions = model.predict(X_test) 
print(classification_report(y_test, predictions)) 



              precision    recall  f1-score   support

          -1       0.59      0.62      0.61        16
           0       0.00      0.00      0.00         6
           1       0.70      0.82      0.75        28

    accuracy                           0.66        50
   macro avg       0.43      0.48      0.45        50
weighted avg       0.58      0.66      0.62        50



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'gamma':['scale', 'auto'],
              'kernel': ['linear']}  
   
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, grid_predictions)) 

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 4/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.595 total time=   0.0s
[CV 4/5] END .C=0.1, gamma=scale, kernel=linear;, score=0.595 total time=   0.0s
[CV 1/5] END .C=0.1, gamma=scale, kernel=linear;, score=0.465 total time=   0.0s
[CV 5/5] END .C=0.1, gamma=scale, kernel=linear;, score=0.595 total time=   0.0s
[CV 1/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.465 total time=   0.0s
[CV 3/5] END .C=0.1, gamma=scale, kernel=linear;, score=0.524 total time=   0.0s
[CV 2/5] END .C=0.1, gamma=scale, kernel=linear;, score=0.605 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.605 total time=   0.0s
[CV 3/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.524 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.595 total time=   0.0s
[CV 1/5] END ...C=1, gamma=scale, kernel=linear;, score=0.512 total time=   0.0s
[CV 3/5] END ...C=1, gamma=scale, kernel=linear;,

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [99]:
a= X_test[:]
a=a.head(1)

b = table.loc[table['Team'] == 'Chelsea']
c =  table.loc[table['Team'] == 'Liverpool']
a.AAS = c.AAS.values[0]
a.ADS = c.ADS.values[0]
a.HAS = b.HAS.values[0]
a.HDS = b.HDS.values[0]
a.AR=2

grid.predict(a)

array([1])

In [20]:
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train,y_train)

#use model to make predictions on test data
y_pred = logistic_regression.predict(X_test)
print(classification_report(y_test, y_pred))
y_pred

              precision    recall  f1-score   support

          -1       0.62      0.62      0.62        16
           0       0.00      0.00      0.00         6
           1       0.71      0.86      0.77        28

    accuracy                           0.68        50
   macro avg       0.44      0.49      0.47        50
weighted avg       0.60      0.68      0.63        50



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([ 1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1,  1, -1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1])

In [66]:
a= X_test[:]
a=a.head(1)

b = table.loc[table['Team'] == 'Chelsea']
c =  table.loc[table['Team'] == 'Liverpool']
a.AAS = c.AAS.values[0]
a.ADS = c.ADS.values[0]
a.HAS = b.HAS.values[0]
a.HDS = b.HDS.values[0]

#logistic_regression.predict(a)

In [21]:
from sklearn.ensemble import RandomForestRegressor

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)

# fit your model
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(a)
rf_pred

array([-0.06])

In [73]:
rf_model = RandomForestRegressor(random_state=1)

# fit your model
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(X_test)
rf_pred





array([ 0.19,  0.59,  0.84,  0.02, -0.98,  0.63, -0.6 ,  0.11,  0.28,
        0.26, -0.1 , -0.67, -0.87,  0.72,  0.3 ,  0.35,  0.15,  0.13,
        0.21,  0.28, -0.56, -0.2 , -0.33,  0.11, -0.26, -0.23,  0.33,
        0.31, -0.32,  0.1 , -0.25, -0.71,  0.08,  0.49,  0.7 ,  0.94,
       -0.08,  0.73,  0.53,  0.29,  0.79,  0.37,  0.87,  0.44,  0.03,
        0.19,  0.34,  0.32, -0.42,  0.61, -0.84, -0.49, -0.49,  0.37,
        0.72, -0.16,  0.47,  0.28,  0.5 , -0.51,  0.  ,  0.32,  0.75,
        0.18,  0.24, -0.53,  0.5 ,  0.05,  0.74,  0.56,  0.06,  0.1 ,
        0.71, -0.6 ,  0.65])

In [74]:

y_test

55     1
144    1
249    1
123    1
108    0
      ..
50     0
2      0
80     1
65     1
89     1
Name: Result, Length: 75, dtype: int64

0.37